## Steps to be Followed
1. Importing important libraries
2. Creating S3 bucket
3. Mapping train And Test data in S3
4. Mapping the path of the models in S3

In [70]:
import sagemaker
import boto3 #acess the S3 bucket form local server
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session

In [71]:
bucket_name= "Examplebankapplication" 
my_region= boto3.session.Session().region_name
print(my_region)

us-east-2


In [85]:
from sagemaker.amazon.amazon_estimator import get_image_uri

In [72]:
#accesing bucket
s3= boto3.resource("s3")
try:
    if my_region=="us-east-2":
        s3.create_bucket(Bucket=bucket_name)
    print("S3 bucket created successfully")
except Exception as e:
    print("S3 error:",3)

S3 error: 3


In [73]:
import boto3
s3 = boto3.client('s3')
s3.create_bucket(Bucket='examplebankapplication',CreateBucketConfiguration={'LocationConstraint': 'us-east-2'})

BucketAlreadyOwnedByYou: An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.

### Bucket=bucket_name use this for namimg properly bucket name as output path

In [74]:
prefix= "xgboost-as-a-built-in-algo"
output_path= "s3://{}/{}/output".format("examplebankapplication",prefix)
print(output_path)

s3://examplebankapplication/xgboost-as-a-built-in-algo/output


In [91]:
output_path

's3://examplebankapplication/xgboost-as-a-built-in-algo/output'

### Downloading the Dataset and storing in S3

In [20]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)
    
    
try:
    model_data= pd.read_csv('./bank_clean.csv',index_col=0)
    print("success")
except Exception as e:
    print("data load error",e)

Success: downloaded bank_clean.csv.
success


In [22]:
## Train Test split
import numpy as np
train_data, test_data= np.split(model_data.sample(frac=1,
    random_state=1729),[int(0.7*len(model_data))])
print(train_data.shape,test_data.shape)

(28831, 61) (12357, 61)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [24]:
## Saving Train and Test data in Buckets
## We start with train data
## In aws sagemaker the format is to have dependent variable as first 
## and concenate other variables
pd.concat([train_data['y_yes'],train_data.drop(['y_no','y_yes'],
                                              axis=1)],
         axis=1).to_csv('train_csv',index=False,header=False)

In [28]:
import os

- this Train_csv will get saved in as file in homepage now we need to take this file and upload in S3 bucket, below thats what we are doing

In [31]:
boto3.Session().resource("s3").Bucket(bucket_name).Object(os.path.join(prefix,'train/train_csv')).upload_file('train_csv')

- when we are going to train the model the data path will given by S3 bucket for that we have to create that path

AttributeError: module 'sagemaker' has no attribute 's3_input'

In [33]:
# Test Data Into Buckets
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test_csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test_csv')).upload_file('test_csv')


AttributeError: module 'sagemaker' has no attribute 's3_input'

In [34]:
s3_input_train= sagemaker.TrainingInput(s3_data='s3://{}/{}/train'
                                  .format(bucket_name,prefix),
                                  content_type='csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

In [93]:
s3_input_test 

## Builing Models Xgboost- inbuilt Algorithm


In [47]:
region_name='us-east-2'

In [81]:
container = sagemaker.image_uris.retrieve("xgboost", boto3.Session().region_name,version ="latest")

In [82]:
container

'825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest'

In [83]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "verbosity":"1",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [98]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          use_spot_instances=True,
                                          max_run=300,
                                          max_wait=600)

In [99]:
estimator.fit({'train':s3_input_train,'validation':s3_input_test})

INFO:sagemaker:Creating training-job with name: xgboost-2023-12-17-01-26-35-774


2023-12-17 01:26:35 Starting - Starting the training job...
2023-12-17 01:26:50 Starting - Preparing the instances for training......
2023-12-17 01:27:56 Downloading - Downloading input data...
2023-12-17 01:28:26 Downloading - Downloading the training image...
2023-12-17 01:28:42 Training - Training image download completed. Training in progress..Arguments: train
[2023-12-17:01:29:00:INFO] Running standalone xgboost training.
[2023-12-17:01:29:00:INFO] File size need to be processed in the node: 4.83mb. Available memory size in the node: 23952.12mb
[2023-12-17:01:29:00:INFO] Determined delimiter of CSV input is ','
[01:29:00] S3DistributionType set as FullyReplicated
[01:29:00] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2023-12-17:01:29:00:INFO] Determined delimiter of CSV input is ','
[01:29:00] S3DistributionType set as FullyReplicated
[01:29:00] 12357x59 matrix with 729063 entries loaded from /opt/ml/input/data/v

## Deploy Machine learning Model

In [100]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: xgboost-2023-12-17-17-31-40-233
INFO:sagemaker:Creating endpoint-config with name xgboost-2023-12-17-17-31-40-233
INFO:sagemaker:Creating endpoint with name xgboost-2023-12-17-17-31-40-233


-----!

### Prediction if the Test Data

In [104]:
from sagemaker.serializers import CSVSerializer

In [110]:
test_data_array= test_data.drop(['y_no','y_yes'], axis=1).values #load the data into an array
xgb_predictor.content_type= 'text/csv' #data type for inference
xgb_predictor.serializer = CSVSerializer() #set the serializer type


In [115]:
predictions= xgb_predictor.predict(test_data_array).decode('utf-8') # predict
prediction_array= np.fromstring(predictions[1:],sep=',') # turn prediction u=into array
print(prediction_array.shape)

(12357,)


In [116]:
prediction_array

array([0.05578489, 0.04585116, 0.05088526, ..., 0.03450042, 0.03321051,
       0.03635123])

## Creating confusion matrix taken from AWS module

In [118]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(prediction_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.5%

Predicted      No Purchase    Purchase
Observed
No Purchase    90% (10784)    36% (152)
Purchase        10% (1149)     64% (272) 



In [125]:
bucket_name='examplebankapplication'

## Deleting the the endpoints

In [126]:
#sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete= boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': 'GQHC8V5XX4H2NA53',
   'HostId': 'brAF3oS6FpVNiRF4be5m2doJcQ+Y1ZDi80+1EIe8MAwBhsz9h4gVornXqufTpUm0wUN2OkxJxv4=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'brAF3oS6FpVNiRF4be5m2doJcQ+Y1ZDi80+1EIe8MAwBhsz9h4gVornXqufTpUm0wUN2OkxJxv4=',
    'x-amz-request-id': 'GQHC8V5XX4H2NA53',
    'date': 'Sun, 17 Dec 2023 17:57:11 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/xgboost-2023-12-17-01-26-35-774/profiler-output/system/incremental/2023121701/1702776480.algo-1.json'},
   {'Key': 'xgboost-as-a-built-in-algo/test/test_csv'},
   {'Key': 'xgboost-as-a-built-in-algo/output/xgboost-2023-12-17-01-26-35-774/profiler-output/system/training_job_end.ts'},
   {'Key': 'xgboost-as-a-built-in-algo/output/xgboost-2023-12-17-01-26-35-774/debug-output/training_job_end.ts'},
   {'Key': 'xgb